In [ ]:
# !pip install pandas
# !pip install torch
# !pip install tqdm

In [1]:
import torch
from dna_dataset import *
from constants import *
# import pandas as pd

Unzip the datafile

In [2]:
!unzip $DATA_ZIP_FILE

Archive:  Files.zip
  inflating: Files/accessible.fasta  
  inflating: Files/notaccessible.fasta  
  inflating: Files/test.fasta        


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
train_dataset = DNADataset(ACCESSSIBLE_FILE, NOT_ACCESSIBLE_FILE)

In [4]:
# ensure the DNADataset is loaded properly
print('total sequences', len(train_dataset.sequences))
print('num accessible', train_dataset.accessible_count)
print('num not accessible', train_dataset.not_accessible_count)
i = 0
print(f"example entry {i}")
item = train_dataset[i]
print("label", item['label'])
print(item['sequence'])

# ensure dataset was shuffled properly
# check that not all the accessible labels are at the front
for i in range(train_dataset.accessible_count):
    if train_dataset[i]['label'] != ACCESSIBLE_LABEL:
        print('shuffled')
        break

total sequences 525688
num accessible 47239
num not accessible 478449
example entry 0
label 0
GAAATAATACCTATTGGGTTGCTTGAACCCGGGTTTTCATTTTATGATGC
TAATTTTATTATACTGCACAGAAGCAGATTCATCTAATTCAGGAGCAACT
TCATGTGTTATATCTGCAGGATGTACATAGTCAGTTGTTACAACATTATC
TCTCTCCTGAGAAACATAGGCAAAGATTCCATGAAAGAAAATTTCTGCAG
shuffled


In [8]:
from tqdm import tqdm

batch_size = 32

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True
)
# val_loader = torch.utils.data.DataLoader(
#     val_dataset, batch_size=batch_size, shuffle=True
# )

# for batch in tqdm(train_loader):  # show the times for each batch
#     # Forward propagate
#     samples, labels = batch['input_ids'], batch['labels']

#     samples.to(device)
#     labels.to(device)
#     outputs = model(samples)